<img src="https://raw.githubusercontent.com/Sengsathit/OCR_data_scientist_assets/main/header_pret_a_depenser.png" alt="Alternative text" />

# Introduction

# Imports

In [24]:
import warnings
import numpy as np
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV, train_test_split

import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

import lightgbm as lgb
import gc

import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')

# Configurer Pandas pour un affichage complet du contenu des colonnes
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Chargement des données

In [25]:
df_train = pd.read_csv('../../datasets/df_train_domain.csv')
df_test = pd.read_csv('../../datasets/df_test_domain.csv')

labels = df_train['TARGET']

# Modélisation

## MLflow

In [26]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

## Pre-processing

In [27]:
# Supprimer la cible des données d'entraînement
if 'TARGET' in df_train:
    train = df_train.drop(columns = ['TARGET'])
else:
    train = df_train.copy()

# Noms des caractéristiques
features = list(train.columns)

# Copie des données de test
test = df_test.copy()

# Imputation médiane des valeurs manquantes
imputer = SimpleImputer(strategy = 'median')

# Normaliser chaque caractéristique entre 0 et 1
scaler = MinMaxScaler(feature_range = (0, 1))

# Ajuster sur les données d'entraînement
imputer.fit(train)

# Transformer à la fois les données d'entraînement et de test
train = imputer.transform(train)
test = imputer.transform(df_test)

# Transformer avec le scaler
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

print('Forme des données d\'entraînement : ', train.shape)
print('Forme des données de test : ', test.shape)

Forme des données d'entraînement :  (307511, 244)
Forme des données de test :  (48744, 244)


In [28]:
# Datasets d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size=0.2, random_state=42)

## Logistic Regression

In [31]:
# Créer le modèle avec le paramètre de régularisation spécifié
logistic_regression = LogisticRegression(solver='liblinear')

# Définir la grille d'hyperparamètres
params_grid = {'C': [0.0001, 0.001, 0.01]}

# Configuration de GridSearchCV, avec 'roc_auc' comme métrique de scoring
grid_search = GridSearchCV(logistic_regression, params_grid, cv=5, scoring='roc_auc', return_train_score=True)

# Exécuter GridSearchCV
grid_search.fit(X_train, y_train)

# Extraire le meilleur modèle, ses hyperparamètres et son score AUC
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
best_auc = grid_search.best_score_

# Session MLflow
mlflow.set_experiment("Logistic Regression")

# Démarrer une nouvelle exécution pour le meilleur modèle
with mlflow.start_run(run_name="Run"):
    # Loguer les hyperparamètres du meilleur modèle
    mlflow.log_param('C', best_params['C'])

    # Calculer et loguer l'AUC pour le train et le test
    y_pred_train_proba = best_model.predict_proba(X_train)[:, 1]
    y_pred_test_proba = best_model.predict_proba(X_test)[:, 1]
    auc_train = roc_auc_score(y_train, y_pred_train_proba)
    auc_test = roc_auc_score(y_test, y_pred_test_proba)
    
    mlflow.log_metric('auc_train', auc_train)
    mlflow.log_metric('auc_test', auc_test)

    # Loguer le modèle pour la meilleure configuration
    mlflow.sklearn.log_model(
        sk_model=best_model,
        artifact_path='logistic_regression_best_model_path',
        signature=infer_signature(X_train, best_model.predict(X_train)),
        input_example=X_train[:1],
        registered_model_name="logistic_regression_best_model"
    )

Registered model 'logistic_regression_best_model' already exists. Creating a new version of this model...
2024/09/05 13:42:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: logistic_regression_best_model, version 3
Created version '3' of model 'logistic_regression_best_model'.
2024/09/05 13:42:51 INFO mlflow.tracking._tracking_service.client: 🏃 View run Run at: http://127.0.0.1:8080/#/experiments/407899967081860444/runs/a66e09d50143416da0ff43ddf521723d.
2024/09/05 13:42:51 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/407899967081860444.


## Random Forest

In [ ]:
df_train = df_train.drop(columns = 'TARGET')

features_name = list(df_train.columns)

# Imputer les valeurs manquantes
imputer = SimpleImputer(strategy = 'median')
features_train = imputer.fit_transform(df_train)
features_test = imputer.transform(df_test)

# Normaliser les caractéristiques
scaler = MinMaxScaler(feature_range = (0, 1))
features_train = scaler.fit_transform(features_train)
features_test = scaler.transform(features_test)

# Instance de modèle Random Forest Classifier
random_forest_classifier = RandomForestClassifier(n_estimators = 100, random_state = 50, verbose = 1, n_jobs = -1)

# Entraîner le modèle sur les données d'entraînement
random_forest_classifier.fit(features_train, train_labels)

# Extraire les feature importances
feature_importance_values = random_forest_classifier.feature_importances_
feature_importances = pd.DataFrame({'feature': features_name, 'importance': feature_importance_values})

# Faire des prédictions sur les données de test
predictions = random_forest_classifier.predict_proba(features_test)[:, 1]

In [ ]:
# DataFrame des prédictions
submit = df_test[['SK_ID_CURR']]
submit['TARGET'] = predictions

# Sauvegarder les prédictions
submit.to_csv('../../datasets/output/random_forest_baseline_domain.csv', index = False)

### Interprétation du modèle : Feature Importances

In [ ]:
def plot_feature_importances(df):
    """
    Trace les importances retournées par un modèle.
    
    Args:
        df (dataframe) : importances des caractéristiques. Doit contenir les 
        caractéristiques dans une colonne appelée `features` et les importances 
        dans une colonne appelée `importance`.
        
    Returns:
        Affiche un graphique des 15 caractéristiques les plus importantes.
        
        df (dataframe) : importances des caractéristiques triées par importance 
        (de la plus élevée à la plus faible) avec une colonne pour l'importance normalisée.
    """
    
    # Trier les caractéristiques en fonction de leur importance
    df = df.sort_values('importance', ascending=False).reset_index()
    
    # Normaliser les importances des caractéristiques pour qu'elles totalisent un
    df['importance_normalized'] = df['importance'] / df['importance'].sum()

    # Créer un graphique à barres horizontales des importances des caractéristiques
    plt.figure(figsize=(10, 6))
    ax = plt.subplot()
    
    # Il est nécessaire d'inverser l'index pour afficher les plus importantes en haut
    ax.barh(list(reversed(list(df.index[:15]))), 
            df['importance_normalized'].head(15), 
            align='center', edgecolor='k')
    
    # Définir les graduations et étiquettes sur l'axe y
    ax.set_yticks(list(reversed(list(df.index[:15]))))
    ax.set_yticklabels(df['feature'].head(15))
    
    # Étiquetage du graphique
    plt.xlabel('Importance Normalisée'); plt.title('Feature Importances')
    plt.show()
    
    return df

In [ ]:
feature_importances.head()

In [ ]:
# Affichage des feature importances
feature_importances_sorted = plot_feature_importances(feature_importances)

## Light Gradient Boosting Machine